In [5]:
import pandas as pd
import numpy as np
import random

In [6]:
customers = pd.read_csv("train.csv")
#customers = customers.drop(["Ever_Married", "Age", "Graduated", "Work_Experience", "Family_Size", "Var_1", "Segmentation"], axis=1)
customers

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [7]:
df1 = pd.read_csv("Train.csv.xls")
df1

FileNotFoundError: [Errno 2] No such file or directory: 'Train.csv.xls'

In [ ]:
df = pd.merge(customers, df1, left_index=True, right_index=True)
df = df.loc[:, ["Gender_x", "Age", "Profession","Work_Experience","Family_Size", "Spending_Score", "Cost_of_the_Product", "Prior_purchases", "Discount_offered"]]
df = df.dropna(subset="Profession")
df["Work_Experience"] = df["Work_Experience"].fillna(0.0)
df["Family_Size"] = df["Family_Size"].fillna(0.0)
df.reset_index(inplace=True, drop=True)
df["Family_Size"].max()

NameError: name 'customers' is not defined

In [ ]:
df.to_csv("customers.csv", index=False)
professions = df["Profession"].unique()
professions

array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
       'Executive', 'Doctor', 'Homemaker', 'Marketing'], dtype=object)

In [10]:
df = pd.read_csv("customers.csv")

In [12]:
laptops = pd.read_csv("laptops.csv")
laptops = laptops.loc[:, ["Laptop", "Brand", "Final Price"]]

professions = df["Profession"].unique()
laptops["Profession"] = laptops.apply(lambda row: ", ".join([str(i) for i in random.sample(list(professions), np.random.choice(range(1, 3)))]), axis=1)
max_price = laptops["Final Price"].max()
min_price = laptops["Final Price"].min()
laptops["Normalized"] = (laptops["Final Price"]-min_price) / (max_price - min_price)
laptops.rename(columns={"Final Price":"Price"}, inplace=True)
laptops["Price"].mean()

1312.638509259259

In [ ]:
keyboards = pd.read_csv("all_keyboards.csv")
keyboards = keyboards.iloc[:, [1, 2]]
keyboards["Profession"] = keyboards.apply(lambda row: ", ".join([str(i) for i in random.sample(list(professions), np.random.choice(range(1, 3)))]), axis=1)
max_price = keyboards["Price"].max()
min_price = keyboards["Price"].min()
keyboards["Normalized"] = (keyboards["Price"]-min_price) / (max_price - min_price)
keyboards.rename(columns={"Price":"Price"}, inplace=True)
keyboards

,Name,Price,Profession,Normalized
0,Philips K264,7.50,"Engineer, Executive",0.011938
1,Philips SPT6324 Black,9.99,Doctor,0.018222
2,Philips SPT6224 Black,8.93,Marketing,0.015547
3,Logitech MX Keys,106.44,"Entertainment, Healthcare",0.261641
4,Powertech PT-677,3.99,Lawyer,0.003079
...,...,...,...,...
1265,Esperanza TK105,6.80,Entertainment,0.010171
1266,Keep Out F90E Gaming USB,39.90,"Homemaker, Executive",0.093708
1267,Gigabyte K8100,75.00,"Marketing, Doctor",0.182293
1268,Approx Quasar,29.00,"Engineer, Homemaker",0.066199


In [ ]:
laptops.to_csv("laptops_data.csv", index=False)
keyboards.to_csv("keyboards_data.csv", index=False)

In [ ]:
from dataclasses import dataclass

@dataclass
class Customer:
    def __init__(self, funds: int = None, possible_discount: float = None, profession: str = None, power: int = None):
        self.funds = funds
        self.possible_discount = possible_discount
        self.profession = profession
        self.power = power

def recommend(customer: Customer):
    multiplier = (1, 1)
    match customer.power:
        case 1:
            multiplier = (1, 1.5)
        case 2:
            multiplier = (1.5, 2.5)
        case 3: 
            multiplier = (2.5, np.inf)
    
    predictions = laptops[laptops["Profession"] == customer.profession]
    predictions = predictions[predictions["Final Price"] <= customer.funds * multiplier[1]]
    predictions = predictions[predictions["Final Price"] >= customer.funds * multiplier[0]]

    pred_key = keyboards[keyboards["Profession"] == customer.profession]
    pred_key = pred_key.sort_values(by="Price")
    
    mean = pred_key["Price"].mean()

    quantile = 0.5
    match customer.power:
        case 1:
            quantile = 1/3
            quant = np.quantile(pred_key["Price"], quantile)
            pred_key = pred_key[pred_key["Price"] < mean-quant]
        case 2:
            quantile = 1/3
            quant = np.quantile(pred_key["Price"], quantile)
            pred_key = pred_key[pred_key["Price"] > mean-quant]
            quantile = 2/3
            quant = np.quantile(pred_key["Price"], quantile)
            pred_key = pred_key[pred_key["Price"] < mean+quant]
        case 3: 
            quantile = 2/3
            quant = np.quantile(pred_key["Price"], quantile)
            pred_key = pred_key[pred_key["Price"] > mean+quant]
    
    return predictions, pred_key

def recommend2(customer: Customer):
    
    return predictor(customer, laptops, "Final Price"), predictor(customer, keyboards, "Price")

def predictor(customer: Customer, df: pd.DataFrame, column: str):
    pred_key = df[df["Profession"] == customer.profession]
    pred_key = pred_key.sort_values(by=column)
    mean = pred_key[column].mean()

    print

    quantile = 0.5
    match customer.power:
        case 1:
            quantile = 1/3
            quant = np.quantile(pred_key[column], quantile)
            pred_key = pred_key[pred_key[column] < mean-quant]
        case 2:
            quantile = 1/3
            quant = np.quantile(pred_key[column], quantile)
            pred_key = pred_key[pred_key[column] > mean-quant]
            quantile = 2/3
            quant = np.quantile(pred_key[column], quantile)
            pred_key = pred_key[pred_key[column] < mean+quant]
        case 3: 
            quantile = 2/3
            quant = np.quantile(pred_key[column], quantile)
            pred_key = pred_key[pred_key[column] > mean+quant]
    return pred_key

recommend2(Customer(funds=200, profession="Engineer", power=2))[0]

,Laptop,Brand,Final Price,Profession,Normalized
1395,HP 15S-FQ4053NS Intel Core i5-1155G7/8GB/256GB...,HP,542.99,Engineer,0.049204
1578,Lenovo V15 G3 ABA AMD Ryzen 5 5625U/8GB/512GB ...,Lenovo,547.63,Engineer,0.049872
149,ASUS ExpertBook P1512CEA-EJ0083X Intel Core i3...,Asus,549.00,Engineer,0.050069
851,HP 250 G8 59T50EA Intel Core i3-1115G4/8GB/512...,HP,549.00,Engineer,0.050069
439,ASUS Vivobook 14 F1400EA-EK1543W Intel Core i3...,Asus,549.00,Engineer,0.050069
...,...,...,...,...,...
596,Lenovo Legion Pro 7 16IRX8 Intel Core i9-13900...,Lenovo,2689.01,Engineer,0.358010
2109,MSI Raider GE76 12UGS-005XES Intel Core i7-127...,MSI,2749.00,Engineer,0.366642
199,Razer Blade 15 QHD Intel Core i7-12800H/16GB/1...,Razer,2799.99,Engineer,0.373979
916,Razer Blade 15 Advanced Model QHD Intel Core i...,Razer,2800.00,Engineer,0.373981


In [ ]:
class ProductRecommender:
    def __init__(self, laptops, keyboards, customer: Customer=None):
        self.laptops = laptops
        self.keyboards = keyboards
        self.customer = customer
    
    def update_customer(self, customer: Customer):
        if customer is None:
            raise Exception("Customer cannnot be none")
        self.customer = customer

    def recommend(self):
        if self.customer is None:
            raise Exception("Customer must be defeined, use update_customer")

        return self._filter_by_recommendation("Final Price", self.laptops), self._filter_by_recommendation("Price", self.keyboards)
    
    def _filter_by_recommendation(self, columm: str, df: pd.DataFrame):
        pred = df[df["Profession"] == self.customer.profession]
        pred = pred.sort_values(by=columm)
        
        mean = pred[str].mean()

        quantile = 0.5
        match self.customer.power:
            case 1:
                quantile = 1/3
                quant = np.quantile(pred[columm], quantile)
                pred = pred[pred[columm] < mean-quant]
            case 2:
                quantile = 1/3
                quant = np.quantile(pred[columm], quantile)
                pred = pred[pred[columm] > mean-quant]
                quantile = 2/3
                quant = np.quantile(pred[columm], quantile)
                pred = pred[pred[columm] < mean+quant]
            case 3: 
                quantile = 2/3
                quant = np.quantile(pred[columm], quantile)
                pred = pred[pred[columm] > mean+quant]
        
        return pred


In [ ]:
from math import ceil
max_funds = df["Cost_of_the_Product"].max()
customer = Customer(funds=0.012, power=ceil(0.5548*3), profession="Engineer")
recommend2(customer)[0]

,Laptop,Brand,Final Price,Profession,Normalized
1117,Acer Extensa 15 EX215-54-54AL Intel Core i5-11...,Acer,709.65,Engineer,0.073186
1337,Dell Vostro 3420 Intel Core i5-1135G7/8 GB/256...,Dell,717.00,Engineer,0.074244
1997,Lenovo Yoga Creator 7 15IMH05 Intel Core i7-10...,Lenovo,786.84,Engineer,0.084293
1983,Lenovo ThinkPad T580 Intel Core i5-8250U/8GB/2...,Lenovo,802.00,Engineer,0.086475
1948,Lenovo IdeaPad Intel Core i5-1135G7/8GB/512GB ...,Lenovo,802.61,Engineer,0.086563
112,ASUS VivoBook F1605PA-MB124W Intel Core i7-113...,Asus,819.00,Engineer,0.088921
1782,Asus ROG Strix G15 G512LW-HN038 Intel Core i7-...,Asus,1106.34,Engineer,0.130268
1624,Microsoft Surface Laptop 4 Negro Intel Core i7...,Microsoft,1251.19,Engineer,0.151112
1309,Deep Gaming Nubian 15 Intel Core i7-13700H/32G...,Deep Gaming,1525.00,Engineer,0.190512
2083,MSI Prestige 14 A11SCX-002ES Intel Core i7-118...,MSI,1599.00,Engineer,0.201161


In [ ]:
customer = Customer(0.465, 0.65, "Artist", 0.45)

def recommend_products(customer: Customer, *args) -> list[pd.DataFrame]:
    if not isinstance(customer, Customer):
        raise ValueError("Input must be a Customer object")

    def filter_products(products):
        st_dev = np.std(products["Normalized"]) * customer.power
        pred_products = products[products["Profession"].str.contains(customer.profession)]
        pred_products = pred_products[(pred_products["Normalized"] > customer.funds - st_dev) & (pred_products["Normalized"] < customer.funds + st_dev)]
        return pred_products
    
    def calculate_discount(products):
        prod_mean = np.mean(products["Price"])
        print(prod_mean)
        products["Discounted"] = products["Price"]-prod_mean
        for index, discounted_price in products["Discounted"].items():
            # Calculate discounted price for the current product
            new_discounted_price = products.at[index, "Price"] - max(discounted_price, 0) * customer.possible_discount
            # Update the value in the DataFrame
            products.at[index, "Discounted"] = new_discounted_price
        products["Discounted"] = products["Discounted"].apply(lambda x: round(x, 2))
        return products


    """pred_laptops = filter_products(laptops)
    pred_keyboards = filter_products(keyboards)"""

    predictions = [filter_products(arg) for arg in args]
    predictions = [calculate_discount(p) for p  in predictions]

    return predictions

products = recommend_products(customer, laptops, keyboards)
products = [products[0].loc[:, ["Laptop", "Price"]], products[1].loc[:, ["Name", "Price"]]]
products[1]

3378.0620000000004
181.25833333333333


,Name,Price
142,Microsoft Surface Pro Signature Type Cover Alc...,174.51
325,Microsoft Surface Pro Signature Type Cover Alc...,179.00
586,Razer Huntsman Elite (Razer Linear Optical),192.85
740,Asus ROG Claymore Core with Numpad,189.90
1128,Asus ROG Strix Scope,182.29
1171,Acer Predator Aethon 500,169.00
